## Meta-analysis:

#### ARIC AA and ARIC EA incident CH ~ Prevalent CH PRS + covars

` Incident CH ~ scale(Prevalent CH PRS) + Age at baseline + Sex + Smoking Status (never vs ever) + 
                    PC1 + PC2 + PC3 + PC4 + PC5 + Batch Effect (including centers and/or Visits)`


In [ ]:
##############################
#### Forest plot
##############################
library(data.table) # version 1.14.6
library(meta) # version 6.2-1
library(grid) # version 4.2.2
library(scales) # version 1.2.1
##############################


# set working directory
setwd("/medpop/esp2/mesbah/projects/ch_progression/aric/gwas/PRS")

In [ ]:

# PRS summary
prs_dat <- fread("ch_prs_std.final_glm.multivariable.incident_ch.2023Jul15.csv", header = T)

table(prs_dat$Dataset)
table(prs_dat$Exposure)
table(prs_dat$Outcome)


### Inverse-variance weighted meta-analysis ####

In [ ]:
#### IVW Meta-analysis
source("/medpop/esp2/mesbah/tools/longitudinal-profiling-of-clonal-hematopoiesis/determinants_of_CH/genetic/ivw_metaAnalysis.R")



#### VAF >=2%

In [ ]:
#### VAF >=2%
prs_dat.vaf2 <- subset(prs_dat, 
                       prs_dat$Dataset %in% 
                         c("AA", "EA"))

### ivw meta-analysis ###
meta_prs.vaf2 <- as.data.frame(matrix(NA, 
                                 nrow = 4, 
                                 ncol = ncol(prs_dat.vaf2)))

names(meta_prs.vaf2) <- names(prs_dat.vaf2)

meta_prs.vaf2$Outcome <- unique(prs_dat.vaf2$Outcome)
meta_prs.vaf2$Dataset <- "Overall (VAF>=2%)"
meta_prs.vaf2$Exposure <- "CH_PRS"


In [ ]:
## 
for(i in 1:nrow(meta_prs.vaf2)){
  tmp <- ivw_meta(b = prs_dat.vaf2$Beta[prs_dat.vaf2$Outcome==meta_prs.vaf2$Outcome[i]], 
                  bSE = prs_dat.vaf2$SE[prs_dat.vaf2$Outcome==meta_prs.vaf2$Outcome[i]])
  meta_prs.vaf2$Beta[i] <- tmp[[1]]
  meta_prs.vaf2$SE[i] <- tmp[[2]]
  meta_prs.vaf2$`t-stat`[i] <- tmp[[3]]
  meta_prs.vaf2$P[i] <- tmp[[4]]
}

prs_dat_n_meta.vaf2 <- as.data.frame(rbind(prs_dat.vaf2,
                                            meta_prs.vaf2))

In [ ]:
## Outcome
prs_dat_n_meta.vaf2$OUTCOME[prs_dat_n_meta.vaf2$Outcome=="incident_CH"] <- "Overall CH"
prs_dat_n_meta.vaf2$OUTCOME[prs_dat_n_meta.vaf2$Outcome=="incident_DNMT3A"] <- "DNMT3A"
prs_dat_n_meta.vaf2$OUTCOME[prs_dat_n_meta.vaf2$Outcome=="incident_TET2"] <- "TET2"
prs_dat_n_meta.vaf2$OUTCOME[prs_dat_n_meta.vaf2$Outcome=="incident_ASXL1"] <- "ASXL1"

prs_dat_n_meta.vaf2$Exp[prs_dat_n_meta.vaf2$Dataset=="AA"] <- "ARIC AA"
prs_dat_n_meta.vaf2$Exp[prs_dat_n_meta.vaf2$Dataset=="EA"] <- "ARIC EA"
prs_dat_n_meta.vaf2$Exp[prs_dat_n_meta.vaf2$Exp=="CH_PRS"] <- "Overall"
table(prs_dat_n_meta.vaf2$OUTCOME)
table(prs_dat_n_meta.vaf2$Exp)
####
prs_dat_n_meta.vaf2$color <- "gray"
prs_dat_n_meta.vaf2$color[prs_dat_n_meta.vaf2$Exp=="Overall"] <- "red"
# box shape
prs_dat_n_meta.vaf2$Types <- "square"
prs_dat_n_meta.vaf2$Types[prs_dat_n_meta.vaf2$Exp=="Overall"] <- "diamond"



In [ ]:
### Forest plot
# format 
prs_dat_n_meta.vaf2$P_val <- formatC(x = prs_dat_n_meta.vaf2$P, 
                         digits = 1,
                         format = "E")

# OR
prs_dat_n_meta.vaf2$OR <- formatC(round(exp(prs_dat_n_meta.vaf2$Beta),2), 
                      digits = 2, format = "f")

prs_dat_n_meta.vaf2$lSE <- ( prs_dat_n_meta.vaf2$Beta - 1.96 * prs_dat_n_meta.vaf2$SE)
prs_dat_n_meta.vaf2$uSE <- ( prs_dat_n_meta.vaf2$Beta + 1.96 * prs_dat_n_meta.vaf2$SE)

# 95% CI
prs_dat_n_meta.vaf2$CI95 <- paste0("[",formatC(round(exp( prs_dat_n_meta.vaf2$Beta - 1.96 * prs_dat_n_meta.vaf2$SE),2), 
                                               digits = 2, format = "f"),
                       ", ",
                       formatC(round(exp( prs_dat_n_meta.vaf2$Beta + 1.96 * prs_dat_n_meta.vaf2$SE),2), 
                               digits = 2, format = "f"), 
                       "]")
# (prs_dat_n_meta.vaf2)


In [ ]:
## Sort outcome
prs_dat_n_meta.vaf2$OUTCOME <- ordered(prs_dat_n_meta.vaf2$OUTCOME, 
                                  levels = c("Overall CH", 
                                             "DNMT3A", "TET2", 
                                             "ASXL1"))
## exposure
prs_dat_n_meta.vaf2$Exp <- ordered(prs_dat_n_meta.vaf2$Exp,
                              levels = c("ARIC AA", 
                                         "ARIC EA", 
                                         "Overall"))


In [ ]:
## Format metagen
b_adj_v1 <- metagen(TE = Beta,
                    lower = lSE,
                    upper = uSE,
                    studlab = Exp,
                    subgroup=OUTCOME,
                    data=prs_dat_n_meta.vaf2,
                    sm="OR", 
                    common = F,
                    overall = F, 
                    overall.hetstat = F)

In [ ]:
### adjusted CH, DNMT3A, TET2, ASXL1
pdf("prsCH.final_glm.Forest_incidentCH.2023Jul17.pdf", 
    width = 7, height= 7)
forest(x = b_adj_v1,
       sortvar = prs_dat_n_meta.vaf2$Exp,
       common=F, 
       random=F, 
       hetstat=F, 
       subgroup=k.w>=1, 
       weight.study="common",  
       level=0.95, 
       xlim=c(0.5, 3), 
       smlab="Effect of prevalent CH PRS\non Incident CH\n", 
       smlab.pos=0, 
       colgap=unit(7, "mm"),
       xlab="Odds Ratio (95% CI) for incident CH\nper SD increase in prevalent CH PRS", 
       squaresize=0.6, 
       col.subgroup="black",
       col.square = prs_dat_n_meta.vaf2$color,
       colgap.left=unit(0.1,"cm"),
       colgap.forest.left="3mm", 
       colgap.forest.right="2mm", 
       leftcols=c("studlab"), 
       leftlabs = c("                     "),
       rightcols=c("OR","CI95","P_val"),
       rightlabs=c("OR","95% CI","P"),
       #rightcols=NULL, 
       #rightlabs=NULL,
       col.inside="black", 
       plotwidth=unit(6.5, "cm"), 
       print.subgroup.name=F)
dev.off()


In [ ]:
library(magick)
prs_png <- image_read("prsCH.final_glm.Forest_incidentCH.2023Jul17.png")
gwas_png <- image_read("../GWAS.Forest_incidentCH.v2.28May23.italic.png")


# Resize the images to the same dimensions (if needed)
prs_png <- image_scale(prs_png, "1400x1400")
gwas_png <- image_scale(gwas_png, "2000x1600")

# Combine the two images side by side
combined_image <- image_append(c(prs_png , gwas_png), 
                               stack = F)

# Save the combined image as a new PNG file
image_write(combined_image, 
            "fig4ab.combined_image.side.2023Jul17.png")



#### VAF >=10% ####

In [ ]:
#### VAF >=10%
prs_dat.vaf10 <- subset(prs_dat, 
                       prs_dat$Dataset %in% 
                         c("AA_vaf10", "EA_vaf10")) 

meta_prs.vaf10 <- as.data.frame(matrix(NA, 
                                      nrow = 4, 
                                      ncol = ncol(prs_dat.vaf10)))
names(meta_prs.vaf10) <- names(prs_dat.vaf10)

meta_prs.vaf10$Outcome <- unique(prs_dat.vaf10$Outcome)
meta_prs.vaf10$Dataset <- "Overall (VAF>=10%)"
meta_prs.vaf10$Exposure <- "CH_PRS"


for(i in 1:nrow(meta_prs.vaf10)){
  tmp <- ivw_meta(b = prs_dat.vaf10$Beta[prs_dat.vaf10$Outcome==meta_prs.vaf10$Outcome[i]], 
                  bSE = prs_dat.vaf10$SE[prs_dat.vaf10$Outcome==meta_prs.vaf10$Outcome[i]])
  meta_prs.vaf10$Beta[i] <- tmp[[1]]
  meta_prs.vaf10$SE[i] <- tmp[[2]]
  meta_prs.vaf10$`t-stat`[i] <- tmp[[3]]
  meta_prs.vaf10$P[i] <- tmp[[4]]
}

prs_dat_n_meta.vaf10 <- as.data.frame(rbind(prs_dat.vaf10,
                                            meta_prs.vaf10))


In [ ]:
## Outcome
prs_dat_n_meta.vaf10$OUTCOME[prs_dat_n_meta.vaf10$Outcome=="incident_CH"] <- "Overall CH"
prs_dat_n_meta.vaf10$OUTCOME[prs_dat_n_meta.vaf10$Outcome=="incident_DNMT3A"] <- "DNMT3A"
prs_dat_n_meta.vaf10$OUTCOME[prs_dat_n_meta.vaf10$Outcome=="incident_TET2"] <- "TET2"
prs_dat_n_meta.vaf10$OUTCOME[prs_dat_n_meta.vaf10$Outcome=="incident_ASXL1"] <- "ASXL1"

prs_dat_n_meta.vaf10$Exp[prs_dat_n_meta.vaf10$Dataset=="AA_vaf10"] <- "ARIC AA"
prs_dat_n_meta.vaf10$Exp[prs_dat_n_meta.vaf10$Dataset=="EA_vaf10"] <- "ARIC EA"
prs_dat_n_meta.vaf10$Exp[prs_dat_n_meta.vaf10$Exp=="CH_PRS"] <- "Overall"
table(prs_dat_n_meta.vaf10$OUTCOME)
table(prs_dat_n_meta.vaf10$Exp)
####
prs_dat_n_meta.vaf10$color <- "gray"
prs_dat_n_meta.vaf10$color[prs_dat_n_meta.vaf10$Exp=="Overall"] <- "red"
# box shape
prs_dat_n_meta.vaf10$Types <- "square"
prs_dat_n_meta.vaf10$Types[prs_dat_n_meta.vaf10$Exp=="Overall"] <- "diamond"


In [ ]:
### Forest plot
# format 
prs_dat_n_meta.vaf10$P_val <- formatC(x = prs_dat_n_meta.vaf10$P, 
                         digits = 1,
                         format = "E")

# OR
prs_dat_n_meta.vaf10$OR <- formatC(round(exp(prs_dat_n_meta.vaf10$Beta),2), 
                      digits = 2, format = "f")

prs_dat_n_meta.vaf10$lSE <- ( prs_dat_n_meta.vaf10$Beta - 1.96 * prs_dat_n_meta.vaf10$SE)
prs_dat_n_meta.vaf10$uSE <- ( prs_dat_n_meta.vaf10$Beta + 1.96 * prs_dat_n_meta.vaf10$SE)

# 95% CI
prs_dat_n_meta.vaf10$CI95 <- paste0("[",formatC(round(exp( prs_dat_n_meta.vaf10$Beta - 1.96 * prs_dat_n_meta.vaf10$SE),2), 
                                               digits = 2, format = "f"),
                       ", ",
                       formatC(round(exp( prs_dat_n_meta.vaf10$Beta + 1.96 * prs_dat_n_meta.vaf10$SE),2), 
                               digits = 2, format = "f"), 
                       "]")
# (prs_dat_n_meta.vaf2)


In [ ]:
## Sort outcome
prs_dat_n_meta.vaf10$OUTCOME <- ordered(prs_dat_n_meta.vaf10$OUTCOME, 
                                  levels = c("Overall CH", 
                                             "DNMT3A", "TET2", 
                                             "ASXL1"))
## exposure
prs_dat_n_meta.vaf10$Exp <- ordered(prs_dat_n_meta.vaf10$Exp,
                              levels = c("ARIC AA", 
                                         "ARIC EA", 
                                         "Overall"))


In [ ]:
## Format metagen
b_adj_v2 <- metagen(TE = Beta,
                    lower = lSE,
                    upper = uSE,
                    studlab = Exp,
                    subgroup=OUTCOME,
                    data=prs_dat_n_meta.vaf10,
                    sm="OR", 
                    common = F,
                    overall = F, 
                    overall.hetstat = F)

In [ ]:
### adjusted CH, DNMT3A, TET2, ASXL1
pdf("prsCH.final_glm.Forest_incidentCH.vaf10.2023Jul17.pdf", 
    width = 7, height= 7)
forest(x = b_adj_v2,
       sortvar = prs_dat_n_meta.vaf10$Exp,
       common=F, 
       random=F, 
       hetstat=F, 
       subgroup=k.w>=1, 
       weight.study="common",  
       level=0.95, 
       xlim=c(0.5, 3), 
       smlab="Effect of prevalent CH PRS\non Incident CH\n", 
       smlab.pos=0, 
       colgap=unit(7, "mm"),
       xlab="Odds Ratio (95% CI) for incident CH (VAF>=10%)\nper SD increase in prevalent CH PRS", 
       squaresize=0.6, 
       col.subgroup="black",
       col.square = prs_dat_n_meta.vaf10$color,
       colgap.left=unit(0.1,"cm"),
       colgap.forest.left="3mm", 
       colgap.forest.right="2mm", 
       leftcols=c("studlab"), 
       leftlabs = c("                     "),
       rightcols=c("OR","CI95","P_val"),
       rightlabs=c("OR","95% CI","P"),
       #rightcols=NULL, 
       #rightlabs=NULL,
       col.inside="black", 
       plotwidth=unit(6.5, "cm"), 
       print.subgroup.name=F)
dev.off()
